## <b><font color='darkblue'>Multi-Agent Systems in ADK</font></b>
([source](https://google.github.io/adk-docs/agents/multi-agents/)) <font size='3ptx'><b>As agentic applications grow in complexity, structuring them as a single, monolithic agent can become challenging to develop, maintain, and reason about.</b> The [Agent Development Kit](https://google.github.io/adk-docs/) (ADK) supports building sophisticated applications by composing multiple, distinct <b><font color='blue'>BaseAgent</font></b> instances into a <b><font color='darkblue'>Multi-Agent System</font></b> (MAS).</font>

<b>In ADK, a multi-agent system is an application where different agents, often forming a hierarchy, collaborate or coordinate to achieve a larger goal</b>. Structuring your application this way offers significant advantages, including enhanced modularity, specialization, reusability, maintainability, and the ability to define structured control flows using dedicated workflow agents.

You can compose various types of agents derived from BaseAgent to build these systems:
* <b><font size='3ptx'>LLM Agents</font></b>: Agents powered by large language models. (See [LLM Agents](https://google.github.io/adk-docs/agents/llm-agents/))
* <b><font size='3ptx'>Workflow Agents</font></b>: Specialized agents (<font color='brown'>[SequentialAgent](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/#the-sequentialagent), [ParallelAgent](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/), [LoopAgent](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)</font>) designed to manage the execution flow of their sub-agents. (See [Workflow Agents](https://github.com/johnklee/ml_articles/blob/master/google/agent_development_kit/Workflow_Agent.ipynb))
* <b><font size='3ptx'>Custom agents</font></b>: Your own agents inheriting from BaseAgent with specialized, non-LLM logic. (See [Custom Agents](https://github.com/johnklee/ml_articles/blob/master/google/agent_development_kit/Custom_Agent.ipynb))

The following sections detail the core ADK primitives—such as agent hierarchy, workflow agents, and interaction mechanisms—that enable you to construct and manage these multi-agent systems effectively.

### <b><font color='darkgreen'>1. ADK Primitives for Agent Composition</font></b>
([source](https://google.github.io/adk-docs/agents/multi-agents/#adk-primitives-for-agent-composition)) <font size='3ptx'><b>ADK provides core building blocks — primitives (`sub_agents`)</b> — that enable you to structure and manage interactions within your multi-agent system</font>.

#### <b><font size='3ptx'>1.1. Agent Hierarchy (Parent agent, Sub Agents)</font></b>
The foundation for structuring multi-agent systems is the parent-child relationship defined in <b><font color='blue'>BaseAgent</font></b>.

* <b><font size='3ptx'>Establishing Hierarchy</font></b>: You create a tree structure by passing a list of agent instances to the `sub_agents` argument when initializing a parent agent. ADK automatically sets the `parent_agent` attribute on each child agent during initialization.
* <b><font size='3ptx'>Single Parent Rule</font></b>: An agent instance can only be added as a sub-agent once. Attempting to assign a second parent will result in a `ValueError`.
* <b><font size='3ptx'>Importance</font></b>: This hierarchy defines the scope for [**Workflow Agents**](https://google.github.io/adk-docs/agents/multi-agents/#12-workflow-agents-as-orchestrators) and influences the potential targets for LLM-Driven Delegation. You can navigate the hierarchy using `agent.parent_agent` or find descendants using `agent.find_agent(name)`.

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.0-flash")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.0-flash",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)

# Framework automatically sets:
# assert greeter.parent_agent == coordinator
# assert task_doer.parent_agent == coordinator
```

#### <b><font size='3ptx'>1.2. Workflow Agents as Orchestrators</font></b>
ADK includes specialized agents derived from <b><font color='blue'>BaseAgent</font></b> that don't perform tasks themselves but orchestrate the execution flow of their `sub_agents`.

* <font size='3ptx'>[**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)</font>: Executes its `sub_agents` one after another in the order they are listed.
    - <b>Context</b>: Passes the same [`InvocationContext`](https://google.github.io/adk-docs/runtime/) sequentially, allowing agents to easily pass results via shared state.
 
```python
# Conceptual Example: Sequential Pipeline
from google.adk.agents import SequentialAgent, LlmAgent

step1 = LlmAgent(name="Step1_Fetch", output_key="data") # Saves output to state['data']
step2 = LlmAgent(name="Step2_Process", instruction="Process data from {data}.")

pipeline = SequentialAgent(name="MyPipeline", sub_agents=[step1, step2])
# When pipeline runs, Step2 can access the state['data'] set by Step1.
```

* <font size='3ptx'>[**ParallelAgent**](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/)</font>: Executes its `sub_agents` in parallel. Events from sub-agents may be interleaved.
    - <b>Context</b>: Modifies the `InvocationContext.branch` for each child agent (<font color='brown'>e.g., `ParentBranch.ChildName`</font>), providing a distinct contextual path which can be useful for isolating history in some memory implementations.
    - <b>State</b>: Despite different branches, all parallel children access the same shared `session.state`, enabling them to read initial state and write results (<font color='brown'>use distinct keys to avoid race conditions</font>).

```python
# Conceptual Example: Parallel Execution
from google.adk.agents import ParallelAgent, LlmAgent

fetch_weather = LlmAgent(name="WeatherFetcher", output_key="weather")
fetch_news = LlmAgent(name="NewsFetcher", output_key="news")

gatherer = ParallelAgent(name="InfoGatherer", sub_agents=[fetch_weather, fetch_news])
# When gatherer runs, WeatherFetcher and NewsFetcher run concurrently.
# A subsequent agent could read state['weather'] and state['news'].
```

* <font size='3ptx'>[**LoopAgent**](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)</font>: Executes its sub_agents sequentially in a loop.
    - <b>Termination</b>: The loop stops if the optional `max_iterations` is reached, or if any sub-agent returns an [**Event**](https://google.github.io/adk-docs/events/) with `escalate=True` in it's Event Actions.
    - <b>Context & State</b>: Passes the same [**InvocationContext**](https://google.github.io/adk-docs/runtime/) in each iteration, allowing state changes (<font color='brown'>e.g., counters, flags</font>) to persist across loops.

```python
# Conceptual Example: Loop with Condition
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator

class CheckCondition(BaseAgent): # Custom agent to check state
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        status = ctx.session.state.get("status", "pending")
        is_done = (status == "completed")
        yield Event(author=self.name, actions=EventActions(escalate=is_done)) # Escalate if done

process_step = LlmAgent(name="ProcessingStep") # Agent that might update state['status']

poller = LoopAgent(
    name="StatusPoller",
    max_iterations=10,
    sub_agents=[process_step, CheckCondition(name="Checker")]
)
# When poller runs, it executes process_step then Checker repeatedly
# until Checker escalates (state['status'] == 'completed') or 10 iterations pass.
```

#### <b><font size='3ptx'>1.3. Interaction & Communication Mechanisms</font></b>
Agents within a system often need to exchange data or trigger actions in one another. ADK facilitates this through:

##### <b>a) Shared Session State (`session.state`)</b>
The most fundamental way for agents operating within the same invocation (<font color='brown'>and thus sharing the same [**Session**](https://google.github.io/adk-docs/sessions/session/) object via the [**InvocationContext**](https://google.github.io/adk-docs/runtime/)</font>) to communicate passively.
* <b><font size='3ptx'>Mechanism</font></b>: One agent (or its tool/callback) writes a value (`context.state['data_key'] = processed_data`), and a subsequent agent reads it (`data = context.state.get('data_key')`). State changes are tracked via [**CallbackContext**](https://google.github.io/adk-docs/callbacks/).
* <b><font size='3ptx'>Convenience</font></b>: The `output_key` property on LlmAgent automatically saves the agent's final response text (<font color='brown'>or structured output</font>) to the specified state key.
* <b><font size='3ptx'>Nature</font></b>: Asynchronous, passive communication. Ideal for pipelines orchestrated by [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) or passing data across [**LoopAgent**](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/) iterations.

<b><font color='orange'>Notes: Invocation Context and `temp: State`</font></b>
> When a parent agent invokes a sub-agent, it passes the same `InvocationContext`. This means they share the same temporary (`temp:`) state, which is ideal for passing data that is only relevant for the current turn.

```python
# Conceptual Example: Using output_key and reading state
from google.adk.agents import LlmAgent, SequentialAgent

agent_A = LlmAgent(name="AgentA", instruction="Find the capital of France.", output_key="capital_city")
agent_B = LlmAgent(name="AgentB", instruction="Tell me about the city stored in {capital_city}.")

pipeline = SequentialAgent(name="CityInfo", sub_agents=[agent_A, agent_B])
# AgentA runs, saves "Paris" to state['capital_city'].
# AgentB runs, its instruction processor reads state['capital_city'] to get "Paris".
```

##### <b>b) LLM-Driven Delegation (Agent Transfer)</b>
Leverages an [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/)'s understanding to dynamically route tasks to other suitable agents within the hierarchy.

* <b><font size='3ptx'>Mechanism</font></b>: The agent's LLM generates a specific function call: `transfer_to_agent(agent_name='target_agent_name')`.
* <b><font size='3ptx'>Handling</font></b>: The `AutoFlow`, used by default when sub-agents are present or transfer isn't disallowed, intercepts this call. It identifies the target agent using `root_agent.find_agent()` and updates the [**InvocationContext**](https://google.github.io/adk-docs/runtime/) to switch execution focus.
* <b><font size='3ptx'>Requires</font></b>: The calling [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/) needs clear instructions on when to transfer, and potential target agents need distinct descriptions for the LLM to make informed decisions. Transfer scope (<font color='brown'>parent, sub-agent, siblings</font>) can be configured on the [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/).

```python
# Conceptual Setup: LLM Transfer
from google.adk.agents import LlmAgent

booking_agent = LlmAgent(name="Booker", description="Handles flight and hotel bookings.")
info_agent = LlmAgent(name="Info", description="Provides general information and answers questions.")

coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.0-flash",
    instruction="You are an assistant. Delegate booking tasks to Booker and info requests to Info.",
    description="Main coordinator.",
    # AutoFlow is typically used implicitly here
    sub_agents=[booking_agent, info_agent]
)
# If coordinator receives "Book a flight", its LLM should generate:
# FunctionCall(name='transfer_to_agent', args={'agent_name': 'Booker'})
# ADK framework then routes execution to booking_agent.
```

##### <b>c) Explicit Invocation (AgentTool)</b>
Allows an [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/) to treat another <b><font color='blue'>BaseAgent</font></b> instance as a callable function or [**Tool**](https://google.github.io/adk-docs/tools/).

* <b><font size='3ptx'>Mechanism</font></b>: Wrap the target agent instance in [**AgentTool**](https://google.github.io/adk-docs/tools/function-tools/#agent-tool) and include it in the parent [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/)'s `tools` list. [**AgentTool**](https://google.github.io/adk-docs/tools/function-tools/#agent-tool) generates a corresponding function declaration for the LLM.
* <b><font size='3ptx'>Handling</font></b>: When the parent LLM generates a function call targeting the [**AgentTool**](https://google.github.io/adk-docs/tools/function-tools/#agent-tool), the framework executes `AgentTool.run_async`. This method runs the target agent, captures its final response, forwards any state/artifact changes back to the parent's context, and returns the response as the tool's result.
* <b><font size='3ptx'>Nature</font></b>: Synchronous (<font color='brown'>within the parent's flow</font>), explicit, controlled invocation like any other tool.

```python
# Conceptual Setup: Agent as a Tool
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import agent_tool
from pydantic import BaseModel

# Define a target agent (could be LlmAgent or custom BaseAgent)
class ImageGeneratorAgent(BaseAgent): # Example custom agent
    name: str = "ImageGen"
    description: str = "Generates an image based on a prompt."
    # ... internal logic ...
    async def _run_async_impl(self, ctx): # Simplified run logic
        prompt = ctx.session.state.get("image_prompt", "default prompt")
        # ... generate image bytes ...
        image_bytes = b"..."
        yield Event(author=self.name, content=types.Content(parts=[types.Part.from_bytes(image_bytes, "image/png")]))

image_agent = ImageGeneratorAgent()
image_tool = agent_tool.AgentTool(agent=image_agent) # Wrap the agent

# Parent agent uses the AgentTool
artist_agent = LlmAgent(
    name="Artist",
    model="gemini-2.0-flash",
    instruction="Create a prompt and use the ImageGen tool to generate the image.",
    tools=[image_tool] # Include the AgentTool
)
# Artist LLM generates a prompt, then calls:
# FunctionCall(name='ImageGen', args={'image_prompt': 'a cat wearing a hat'})
# Framework calls image_tool.run_async(...), which runs ImageGeneratorAgent.
# The resulting image Part is returned to the Artist agent as the tool result.
```

These primitives provide the flexibility to design multi-agent interactions ranging from tightly coupled sequential workflows to dynamic, LLM-driven delegation networks. Let's check a real example:
- **`agent_tool_example1`**:

```python
async def roll_dice(n: int) -> int:
  """Roll the dice with n sides.

  Args:
    n: The number of side in a dice to roll.

  Returns:
    The rolling result.
  """
  print(f'Roll the dice with {n} sides...')
  return random.randint(1, n)

async def is_prime(number: int) -> bool:
  """Checks if the given `n` is prime or not.

  Args:
    number: The number to check if it is prime or not.

  Returns:
    True if the given `n` is prime. False otherwise.
  """
  ...


dice_agent = Agent(
    model='gemini-2.0-flash',
    name = "DiceRoll",
    description = (
        "Roll a dice with number from 1 to n and return the obtained number."
    ),
    instruction = "Roll the dice with sides `n` by request.",
    tools=[roll_dice],
)



dice_tool = agent_tool.AgentTool(agent=dice_agent) # Wrap the agent

root_agent = LlmAgent(
    name="GameAgent",
    model="gemini-2.5-pro",
    instruction="You are good at gaming and able to roll a dice.",
    tools=[
        dice_tool,  # Include the AgentTool
        is_prime,
    ]
)
```

In [1]:
import agent_tool_example1

In [2]:
response_text = await agent_tool_example1.call_agent_async('Roll a dice with n as 4')

Roll the dice with 4 sides...


In [3]:
print(response_text)

3



In [6]:
print(await agent_tool_example1.call_agent_async(
    'Roll a dice with n as 10 the first time and n as 1000 the second time. '
    'Sum up them and check if it is a prime. '
    'For the output, show me the process step by step including the value of each rolling. '
    'Like first roll is xxx; second roll is xxx; sum up to xxx which is prime or not.'))

Roll the dice with 10 sides...
Roll the dice with 1000 sides...


First roll is 8; second roll is 802; sum up to 810 which is not a prime.


### <b><font color='darkgreen'>2. Common Multi-Agent Patterns using ADK Primitives</font></b>
<font size='3ptx'>By combining ADK's composition primitives, you can <b>implement various established patterns for multi-agent collaboration</b>.</font>

#### <b><font size='3ptx'>Coordinator/Dispatcher Pattern</font></b>

* <b><font size='3ptx'>Structure</font></b>: A central [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/) (Coordinator) manages several specialized `sub_agents`.
* <b><font size='3ptx'>Goal</font></b>: Route incoming requests to the appropriate specialist agent.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - <b>Hierarchy</b>: Coordinator has specialists listed in `sub_agents`.
    - <b>Interaction</b>: Primarily uses LLM-Driven Delegation (<font color='brown'>requires clear descriptions on sub-agents and appropriate instruction on Coordinator</font>) or Explicit Invocation ([**AgentTool**](https://google.github.io/adk-docs/tools/function-tools/#agent-tool)) (<font color='brown'>Coordinator includes `AgentTool`-wrapped specialists in its `tools`</font>).

```python
# Conceptual Code: Coordinator using LLM Transfer
from google.adk.agents import LlmAgent

billing_agent = LlmAgent(name="Billing", description="Handles billing inquiries.")
support_agent = LlmAgent(name="Support", description="Handles technical support requests.")

coordinator = LlmAgent(
    name="HelpDeskCoordinator",
    model="gemini-2.0-flash",
    instruction="Route user requests: Use Billing agent for payment issues, Support agent for technical problems.",
    description="Main help desk router.",
    # allow_transfer=True is often implicit with sub_agents in AutoFlow
    sub_agents=[billing_agent, support_agent]
)
# User asks "My payment failed" -> Coordinator's LLM should call transfer_to_agent(agent_name='Billing')
# User asks "I can't log in" -> Coordinator's LLM should call transfer_to_agent(agent_name='Support')
```

#### <b><font size='3ptx'>Sequential Pipeline Pattern</font></b>

* <b><font size='3ptx'>Structure</font></b>: A [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) contains `sub_agents` executed in a fixed order.
* <b><font size='3ptx'>Goal</font></b>: Implement a multi-step process where the output of one step feeds into the next.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - <b>Workflow</b>: [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) defines the order.
    - <b>Communication</b>: Primarily uses Shared Session State. Earlier agents write results (often via `output_key`), later agents read those results from `context.state`.

```python
# Conceptual Code: Sequential Data Pipeline
from google.adk.agents import SequentialAgent, LlmAgent

validator = LlmAgent(name="ValidateInput", instruction="Validate the input.", output_key="validation_status")
processor = LlmAgent(name="ProcessData", instruction="Process data if {validation_status} is 'valid'.", output_key="result")
reporter = LlmAgent(name="ReportResult", instruction="Report the result from {result}.")

data_pipeline = SequentialAgent(
    name="DataPipeline",
    sub_agents=[validator, processor, reporter]
)
# validator runs -> saves to state['validation_status']
# processor runs -> reads state['validation_status'], saves to state['result']
# reporter runs -> reads state['result']
```

#### <b><font size='3ptx'>Parallel Fan-Out/Gather Pattern</font></b>

* <b><font size='3ptx'>Structure</font></b>: A [**ParallelAgent**](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/) runs multiple `sub_agents` concurrently, often followed by a later agent (<font color='brown'>in a [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)</font>) that aggregates results.
* <b><font size='3ptx'>Goal</font></b>: Execute independent tasks simultaneously to reduce latency, then combine their outputs.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - [**ParallelAgent**](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/) for concurrent execution (<font color='brown'>Fan-Out</font>). Often nested within a [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) to handle the subsequent aggregation step (<font color='brown'>Gather</font>).
    - <b>Communication</b>: Sub-agents write results to distinct keys in **Shared Session State**. The subsequent "Gather" agent reads multiple state keys.

```python
# Conceptual Code: Parallel Information Gathering
from google.adk.agents import SequentialAgent, ParallelAgent, LlmAgent

fetch_api1 = LlmAgent(name="API1Fetcher", instruction="Fetch data from API 1.", output_key="api1_data")
fetch_api2 = LlmAgent(name="API2Fetcher", instruction="Fetch data from API 2.", output_key="api2_data")

gather_concurrently = ParallelAgent(
    name="ConcurrentFetch",
    sub_agents=[fetch_api1, fetch_api2]
)

synthesizer = LlmAgent(
    name="Synthesizer",
    instruction="Combine results from {api1_data} and {api2_data}."
)

overall_workflow = SequentialAgent(
    name="FetchAndSynthesize",
    sub_agents=[gather_concurrently, synthesizer] # Run parallel fetch, then synthesize
)
# fetch_api1 and fetch_api2 run concurrently, saving to state.
# synthesizer runs afterwards, reading state['api1_data'] and state['api2_data'].
```

#### <b><font size='3ptx'>Hierarchical Task Decomposition</font></b>

* <b><font size='3ptx'>Structure</font></b>: A multi-level tree of agents where higher-level agents break down complex goals and delegate sub-tasks to lower-level agents.
* <b><font size='3ptx'>Goal</font></b>: Solve complex problems by recursively breaking them down into simpler, executable steps.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - <b>Hierarchy</b>: Multi-level `parent_agent`/`sub_agents` structure.
    - <b>Interaction</b>: Primarily LLM-Driven Delegation or Explicit Invocation ([**AgentTool**](https://google.github.io/adk-docs/tools/function-tools/#agent-tool)) used by parent agents to assign tasks to subagents. Results are returned up the hierarchy (<font color='brown'>via tool responses or state</font>).

```python
# Conceptual Code: Hierarchical Research Task
from google.adk.agents import LlmAgent
from google.adk.tools import agent_tool

# Low-level tool-like agents
web_searcher = LlmAgent(name="WebSearch", description="Performs web searches for facts.")
summarizer = LlmAgent(name="Summarizer", description="Summarizes text.")

# Mid-level agent combining tools
research_assistant = LlmAgent(
    name="ResearchAssistant",
    model="gemini-2.0-flash",
    description="Finds and summarizes information on a topic.",
    tools=[agent_tool.AgentTool(agent=web_searcher), agent_tool.AgentTool(agent=summarizer)]
)

# High-level agent delegating research
report_writer = LlmAgent(
    name="ReportWriter",
    model="gemini-2.0-flash",
    instruction="Write a report on topic X. Use the ResearchAssistant to gather information.",
    tools=[agent_tool.AgentTool(agent=research_assistant)]
    # Alternatively, could use LLM Transfer if research_assistant is a sub_agent
)
# User interacts with ReportWriter.
# ReportWriter calls ResearchAssistant tool.
# ResearchAssistant calls WebSearch and Summarizer tools.
# Results flow back up.
```

#### <b><font size='3ptx'>Review/Critique Pattern (Generator-Critic)</font></b>

* <b><font size='3ptx'>Structure</font></b>: Typically involves two agents within a [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/): a Generator and a **Critic/Reviewer**.
* <b><font size='3ptx'>Goal</font></b>: Improve the quality or validity of generated output by having a dedicated agent review it.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - **Workflow**: [**SequentialAgent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) ensures generation happens before review.
    - **Communication**: Shared Session State (<font color='brown'>Generator uses `output_key` to save output; Reviewer reads that state key</font>). The Reviewer might save its feedback to another state key for subsequent steps.

```python
# Conceptual Code: Generator-Critic
from google.adk.agents import SequentialAgent, LlmAgent

generator = LlmAgent(
    name="DraftWriter",
    instruction="Write a short paragraph about subject X.",
    output_key="draft_text"
)

reviewer = LlmAgent(
    name="FactChecker",
    instruction="Review the text in {draft_text} for factual accuracy. Output 'valid' or 'invalid' with reasons.",
    output_key="review_status"
)

# Optional: Further steps based on review_status

review_pipeline = SequentialAgent(
    name="WriteAndReview",
    sub_agents=[generator, reviewer]
)
# generator runs -> saves draft to state['draft_text']
# reviewer runs -> reads state['draft_text'], saves status to state['review_status']
```

#### <b><font size='3ptx'>Iterative Refinement Pattern</font></b>

* <b><font size='3ptx'>Structure</font></b>: Uses a [**LoopAgent**](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/) containing one or more agents that work on a task over multiple iterations.
* <b><font size='3ptx'>Goal</font></b>: Progressively improve a result (<font color='brown'>e.g., code, text, plan</font>) stored in the session state until a quality threshold is met or a maximum number of iterations is reached.
* <b><font size='3ptx'>ADK Primitives Used</font></b>:
    - <b>Workflow</b>: [**LoopAgent**](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/) manages the repetition.
    - <b>Communication</b>: Shared Session State is essential for agents to read the previous iteration's output and save the refined version.
    - <b>Termination</b>: The loop typically ends based on `max_iteration`s or a dedicated checking agent setting `escalate=True` in the `Event Actions` when the result is satisfactory.

```python
# Conceptual Code: Iterative Code Refinement
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator

# Agent to generate/refine code based on state['current_code'] and state['requirements']
code_refiner = LlmAgent(
    name="CodeRefiner",
    instruction="Read state['current_code'] (if exists) and state['requirements']. Generate/refine Python code to meet requirements. Save to state['current_code'].",
    output_key="current_code" # Overwrites previous code in state
)

# Agent to check if the code meets quality standards
quality_checker = LlmAgent(
    name="QualityChecker",
    instruction="Evaluate the code in state['current_code'] against state['requirements']. Output 'pass' or 'fail'.",
    output_key="quality_status"
)

# Custom agent to check the status and escalate if 'pass'
class CheckStatusAndEscalate(BaseAgent):
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        status = ctx.session.state.get("quality_status", "fail")
        should_stop = (status == "pass")
        yield Event(author=self.name, actions=EventActions(escalate=should_stop))

refinement_loop = LoopAgent(
    name="CodeRefinementLoop",
    max_iterations=5,
    sub_agents=[code_refiner, quality_checker, CheckStatusAndEscalate(name="StopChecker")]
)
# Loop runs: Refiner -> Checker -> StopChecker
# State['current_code'] is updated each iteration.
# Loop stops if QualityChecker outputs 'pass' (leading to StopChecker escalating) or after 5 iterations.
```

#### <font size='3ptx'><b>Human-in-the-Loop Pattern</b></font>

* <b><font size='3ptx'>Structure</font></b>: Integrates human intervention points within an agent workflow.
* <b><font size='3ptx'>Goal</font></b>: Allow for human oversight, approval, correction, or tasks that AI cannot perform.
* <b><font size='3ptx'>ADK Primitives Used (Conceptual)</font></b>:
    - <b>Interaction</b>: Can be implemented using a custom **Tool** that pauses execution and sends a request to an external system (<font color='brown'>e.g., a UI, ticketing system</font>) waiting for human input. The tool then returns the human's response to the agent.
    - <b>Workflow</b>: Could use **LLM-Driven Delegation** (`transfer_to_agent`) targeting a conceptual "Human Agent" that triggers the external workflow, or use the custom tool within an [**LlmAgent**](https://google.github.io/adk-docs/agents/llm-agents/#llm-agent).
    - <b>State/Callbacks</b>: State can hold task details for the human; callbacks can manage the interaction flow.
    - <b>Note</b>: ADK doesn't have a built-in "Human Agent" type, so this requires custom integration.

```python
# Conceptual Code: Using a Tool for Human Approval
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.tools import FunctionTool

# --- Assume external_approval_tool exists ---
# This tool would:
# 1. Take details (e.g., request_id, amount, reason).
# 2. Send these details to a human review system (e.g., via API).
# 3. Poll or wait for the human response (approved/rejected).
# 4. Return the human's decision.
# async def external_approval_tool(amount: float, reason: str) -> str: ...
approval_tool = FunctionTool(func=external_approval_tool)

# Agent that prepares the request
prepare_request = LlmAgent(
    name="PrepareApproval",
    instruction="Prepare the approval request details based on user input. Store amount and reason in state.",
    # ... likely sets state['approval_amount'] and state['approval_reason'] ...
)

# Agent that calls the human approval tool
request_approval = LlmAgent(
    name="RequestHumanApproval",
    instruction="Use the external_approval_tool with amount from state['approval_amount'] and reason from state['approval_reason'].",
    tools=[approval_tool],
    output_key="human_decision"
)

# Agent that proceeds based on human decision
process_decision = LlmAgent(
    name="ProcessDecision",
    instruction="Check {human_decision}. If 'approved', proceed. If 'rejected', inform user."
)

approval_workflow = SequentialAgent(
    name="HumanApprovalWorkflow",
    sub_agents=[prepare_request, request_approval, process_decision]
)
```

These patterns provide starting points for structuring your multi-agent systems. You can mix and match them as needed to create the most effective architecture for your specific application.

## <b><font color='darkblue'>Supplement</font></b>
* [**Gemini API Docs - Gemini models**](https://ai.google.dev/gemini-api/docs/models#live-api)